In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
from DL_Studing.datasets.path_dataset import PathDataset
import os

from PIL import Image
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch

In [2]:
with open("data/pixel_path/train/path_info/1.txt", 'r') as f:
    lines = [(int(x), int(y)) for line in f for x, y in [line.strip().split(',')]]
print(lines)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (31, 1), (31, 2), (31, 3), (31, 4), (31, 5), (31, 6), (31, 7), (31, 8), (31, 9), (31, 10), (31, 11), (31, 12), (31, 13), (31, 14), (31, 15), (31, 16), (31, 17), (31, 18), (31, 19), (31, 20), (31, 21), (31, 22), (31, 23), (31, 24), (31, 25), (31, 26), (31, 27), (31, 28), (31, 29), (31, 30), (31, 31)]


In [14]:
x = lines[:-1]
y = lines[1:]
pad = []
for i in range(128 - len(x)):
    pad += [(-1e9, -1e9)]
print(len(x+pad))
print(len(y+pad))

128
128
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (31, 1), (31, 2), (31, 3), (31, 4), (31, 5), (31, 6), (31, 7), (31, 8), (31, 9), (31, 10), (31, 11), (31, 12), (31, 13), (31, 14), (31, 15), (31, 16), (31, 17), (31, 18), (31, 19), (31, 20), (31, 21), (31, 22), (31, 23), (31, 24), (31, 25), (31, 26), (31, 27), (31, 28), (31, 29), (31, 30), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000000000.0, -1000000000.0), (-1000

In [10]:
def cut_patch(img, pos, patch_size=3):
    patches = []
    for coord in pos:
        x, y = coord[0], coord[1]
        # 计算图像块的位置
        if x == -1e9 and y == -1e9:
            patches.append(torch.full((3, patch_size, patch_size), -1e9))
            continue
        x_start = max(x - patch_size // 2, 0)
        x_end = min(x + patch_size // 2 + 1, img.size(1))
        y_start = max(y - patch_size // 2, 0)
        y_end = min(y + patch_size // 2 + 1, img.size(2))

        # 计算边界的补全量
        left_pad = patch_size // 2 if y == 0 else 0
        right_pad = patch_size // 2 if y == img.size(1) - 1 else 0
        bottom_pad = patch_size // 2 if x == img.size(2) - 1 else 0
        top_pad = patch_size // 2 if x == 0 else 0

        # 使用pad函数进行补全
        patch = F.pad(img[:, x_start:x_end, y_start:y_end], (left_pad, right_pad, top_pad, bottom_pad))

        # 将提取的图像块添加到列表中
        patches.append(patch)
    return patches